In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('..')

In [3]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.now()).floor('H')
print(f'{current_date=}')
# current_date = pd.Timestamp('2023-02-28 09:00:00')

current_date=Timestamp('2024-02-14 13:00:00')


In [4]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

/Users/yashwantsaikoneru/opt/anaconda3/envs/conda_environment/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/453781
Connected. Call `.close()` to terminate connection gracefully.


W20240214 13:17:32.019338 79106624 status.cc:137] DoAction result was not fully consumed: Cancelled: Flight cancelled call, with message: CANCELLED. Detail: Cancelled


Finished: Reading data from Hopsworks, using ArrowFlight (14.45s) 
672.0
Earliest date for location ID 1: 2023-09-19 13:00:00+00:00
Latest date for location ID 1: 2023-10-17 12:00:00+00:00


In [5]:
print(features)

     rides_previous_672_hour  rides_previous_671_hour  \
0                        2.0                      2.0   
1                        0.0                      0.0   
2                        0.0                      0.0   
3                        2.0                      4.0   
4                        1.0                      0.0   
..                       ...                      ...   
260                     44.0                     40.0   
261                    101.0                     83.0   
262                    110.0                    111.0   
263                     74.0                     91.0   
264                      5.0                      6.0   

     rides_previous_670_hour  rides_previous_669_hour  \
0                        3.0                      2.0   
1                        0.0                      0.0   
2                        0.0                      0.0   
3                        3.0                      8.0   
4                        0.0  

In [6]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)


#predictions = get_model_predictions(model, features)

In [7]:
model = load_model_from_registry()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/453781
Connected. Call `.close()` to terminate connection gracefully.


In [8]:
predictions = get_model_predictions(model, features)

In [9]:
print(predictions)

     pickup_location_id  predicted_demand
0                     1               1.0
1                     2               0.0
2                     3               0.0
3                     4               3.0
4                     5               0.0
..                  ...               ...
260                 261              38.0
261                 262             102.0
262                 263             106.0
263                 264              60.0
264                 265              10.0

[265 rows x 2 columns]


In [10]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,1.0,2024-02-14 13:00:00
1,2,0.0,2024-02-14 13:00:00
2,3,0.0,2024-02-14 13:00:00
3,4,3.0,2024-02-14 13:00:00
4,5,0.0,2024-02-14 13:00:00
...,...,...,...
260,261,38.0,2024-02-14 13:00:00
261,262,102.0,2024-02-14 13:00:00
262,263,106.0,2024-02-14 13:00:00
263,264,60.0,2024-02-14 13:00:00


In [11]:
from src.inference import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/453781
Connected. Call `.close()` to terminate connection gracefully.


In [12]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 265/265 | Elapsed Time: 00:03 | Remaining Time: 00:00


Launching job: model_predictions_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/453781/jobs/named/model_predictions_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f7feb68ae20>, None)

In [13]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store1 = project.get_feature_store()

# connect to the feature group
feature_group1 = feature_store1.get_feature_group(
    name='model_predictions_feature_group',
    version=1,
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/453781
Connected. Call `.close()` to terminate connection gracefully.


In [15]:
# create feature view (if it doesn't exist yet)
# This feature view only uses on feature group, so the query is trivial

# create feature view if it doesn't exist yet
feature_store1.create_feature_view(
        name=config.FEATURE_VIEW_PREDICTIONS_NAME,
        version=1,
        query=feature_group1.select_all()
    )



RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/453781/featurestores/449604/featureview). Server response: 
HTTP code: 400, HTTP reason: Bad Request, body: b'{"errorCode":270179,"usrMsg":"Feature view: predictions_feature_view, version: 1","errorMsg":"The provided feature view name and version already exists"}', error code: 270179, error msg: The provided feature view name and version already exists, user msg: Feature view: predictions_feature_view, version: 1

In [16]:
import src
project=src.inference.get_hopsworks_project()
store=project.get_feature_store()
feature_group=store.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTIONS,version=1
    )
results=store.get_feature_view(config.FEATURE_VIEW_PREDICTIONS_NAME,version=1)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/453781
Connected. Call `.close()` to terminate connection gracefully.


AttributeError: 'FeatureView' object has no attribute 'head'

In [17]:
p = results.get_batch_data(start_time=current_date-timedelta(days=28),end_time=current_date)

Finished: Reading data from Hopsworks, using ArrowFlight (0.53s) 


In [18]:
print(p)

     pickup_location_id  predicted_demand               pickup_hour
0                   214               0.0 2024-02-12 11:00:00+00:00
1                   128               0.0 2024-02-12 11:00:00+00:00
2                   163             134.0 2024-02-12 11:00:00+00:00
3                   104               0.0 2024-02-12 11:00:00+00:00
4                   149               0.0 2024-02-12 11:00:00+00:00
..                  ...               ...                       ...
260                 220               0.0 2024-02-12 11:00:00+00:00
261                 181               1.0 2024-02-12 11:00:00+00:00
262                  86               0.0 2024-02-12 11:00:00+00:00
263                 253               0.0 2024-02-12 11:00:00+00:00
264                  41              17.0 2024-02-12 11:00:00+00:00

[265 rows x 3 columns]
